# 06_interactive_dashboard.ipynb  
## Interactive Dashboard with ipywidgets, Plotly and Folium

**Objectives:**  
- Build interactive filters to explore crime and weather relationships.  
- Use dropdowns, sliders and map layers for dynamic visualization.  
- Integrate Plotly Express charts and Folium maps in Jupyter.

## 1. Import Libraries and Load Data

In this section, we import all required libraries and load the cleaned crime and weather datasets. We then compute daily crime counts and average temperatures to prepare for visualization.

In [ ]:
# 1. Import libraries and data
import pandas as pd
import plotly.express as px
import folium
from folium.plugins import HeatMap
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output

# Paths
dir_processed = Path('../data/processed')

# Load cleaned data
df_crime = pd.read_csv(dir_processed / 'crime_clean.csv', parse_dates=['date'])
df_weather = pd.read_csv(dir_processed / 'weather_clean.csv', parse_dates=['date'])

# Prepare merged daily dataset
crime_daily = df_crime.groupby('date').size().reset_index(name='crime_count')
if 'temp_max' in df_weather.columns and 'temp_min' in df_weather.columns:
    df_weather['temp_avg'] = (df_weather['temp_max'] + df_weather['temp_min']) / 2
weather_daily = df_weather[['date', 'temp_avg', 'precipitation']]
merged = pd.merge(crime_daily, weather_daily, on='date', how='inner')

# Determine offense column availability
offense_col = 'offense_desc' if 'offense_desc' in df_crime.columns else None

## 2. Define Interactive Widgets

This cell creates a set of ipywidgets controls that allow users to filter the merged dataset by date range, season, and offense type (if available). We configure a `SelectionRangeSlider` for selecting the date window, a `Dropdown` for season selection, a `SelectMultiple` for offense categories, and a button to trigger dashboard updates. All widgets are displayed together in a vertical layout.

In [ ]:
## 2. Define Widgets

# Date range slider
date_slider = widgets.SelectionRangeSlider(
    options=[(d.strftime('%Y-%m-%d'), d) for d in merged['date'].sort_values().unique()],
    index=(0, len(merged['date'].unique())-1),
    description='Date Range',
    orientation='horizontal',
    layout={'width': '800px'}
)

# Seasonal dropdown
dropdown_season = widgets.Dropdown(
    options=['All', 'Winter', 'Spring', 'Summer', 'Fall'],
    value='All',
    description='Season'
)

# Offense multi-select (only if available)
if offense_col:
    offense_types = ['All'] + sorted(df_crime[offense_col].dropna().unique().tolist())
    dropdown_offense = widgets.SelectMultiple(
        options=offense_types,
        value=['All'],
        description='Offense'
    )
else:
    dropdown_offense = None

# Update button
toggle_button = widgets.Button(description='Update Dashboard')

# Display controls
controls = [date_slider, dropdown_season]
if dropdown_offense:
    controls.append(dropdown_offense)
controls.append(toggle_button)
display(widgets.VBox(controls))




## 3. Dashboard Update Function

In this cell, we define the callback function tied to the “Update Dashboard” button. When clicked, it reads the current widget values, filters the merged data accordingly, and regenerates two Plotly charts (crime count over time and crime vs. average temperature). It also builds a Folium heatmap showing crime locations for the selected period. The results are displayed dynamically within an Output widget.

In [ ]:
## 3. Dashboard Update Function

# %%
output = widgets.Output()
@toggle_button.on_click
def update_dashboard(b):
    with output:
        clear_output()
        # Filter by date range
        start, end = date_slider.value
        df_time = merged[(merged['date'] >= start) & (merged['date'] <= end)].copy()

        # Filter by season
        if dropdown_season.value != 'All':
            df_time['month'] = df_time['date'].dt.month
            season_map = {12:'Winter',1:'Winter',2:'Winter',3:'Spring',4:'Spring',5:'Spring',6:'Summer',7:'Summer',8:'Summer',9:'Fall',10:'Fall',11:'Fall'}
            df_time = df_time[df_time['month'].map(season_map) == dropdown_season.value]

        # Filter raw crimes by offense (if applicable)
        df_raw = df_crime.copy()
        if dropdown_offense and 'All' not in dropdown_offense.value:
            df_raw = df_raw[df_raw[offense_col].isin(dropdown_offense.value)]

        # Recompute daily counts after offense filter
        df_counts = df_raw.groupby('date').size().reset_index(name='crime_count')
        df_plot = pd.merge(df_counts, weather_daily, on='date', how='inner')
        df_plot = df_plot[(df_plot['date'] >= start) & (df_plot['date'] <= end)]

        # Plot interactive charts
        fig1 = px.line(df_plot, x='date', y='crime_count', title='Crime Count Over Time')
        fig2 = px.scatter(df_plot, x='temp_avg', y='crime_count', trendline='ols', title='Crime vs Average Temperature')
        display(fig1)
        display(fig2)

        # Folium heatmap
        center = [40.7128, -74.0060]
        m = folium.Map(location=center, zoom_start=11)
        coords = df_raw[(df_raw['date'] >= start) & (df_raw['date'] <= end)][['latitude','longitude']].dropna().values.tolist()
        HeatMap(coords, radius=8).add_to(m)
        display(m)

# Initial output display
display(output)

